In [1]:
import yfinance as yf
from ssk import ssk
from score import get_scores_df
import pandas as pd
from stocksymbol import StockSymbol
from returns import get_monthly_adjusted_price
from pairs import get_pair_distance

In [2]:
ssm = StockSymbol(ssk)

# Get tickers of companies listed in NASDAQ
symbol_list = ssm.get_symbol_list(market="US")
nasdaq_list = [x['symbol'] for x in symbol_list if x['exchange'] == 'NASDAQ']

# Randomly sample 100 of them
nasdaq_list = list(pd.Series(nasdaq_list).sample(100, random_state=1))

In [3]:
# Get the adjusted prices and scores

monthly_adjusted_price = get_monthly_adjusted_price(ticker_list=nasdaq_list, start_year = 2009)
monthly_adjusted_price.to_csv("monthly adjusted price.csv", index=False)

fundamental_scores = get_scores_df(ticker_list=nasdaq_list)
fundamental_scores.to_csv("fundamental scores.csv", index=False)

          CPLP      PAGP      TATT      PCYO      WERN      LYFT      CPOP  \
year                                                                         
2009  1.000000       NaN  1.000000  1.000000  1.000000       NaN       NaN   
2010  1.216509  1.000000  0.570825  1.008009  1.216247       NaN       NaN   
2011  1.404927  1.064082  0.774223  1.001670  1.447323       NaN       NaN   
2012  0.910473  1.124663  1.043000  0.507213  1.451670       NaN       NaN   
2013  0.908259  1.137145  1.183221  1.083464  1.177294       NaN       NaN   
2014  0.841575  1.216564  1.070111  1.122556  1.294267       NaN       NaN   
2015  0.846965  1.490279  1.314823  0.879929  1.919930       NaN       NaN   
2016  0.829709  1.471609  1.057343  1.080148  1.365603  1.000000       NaN   
2017  0.770155  0.887026  1.155193  1.102068  1.990469  1.715065       NaN   
2018  0.635641  1.646309  0.533036  1.135402  2.071311  1.660245  1.000000   
2019  0.672602  1.574787  1.043867  1.181483  2.220731  2.211270

In [7]:
# Closest pairs by price movement

price_pairs = get_pair_distance(monthly_adjusted_price)
fundamental_pairs = get_pair_distance(fundamental_scores)

price_pairs.to_csv("price pairs.csv")
fundamental_pairs.to_csv("fundamental pairs.csv")